In [2]:
import numpy as np
import pymol as cmd
from pymol import cmd,stored
import tensorflow as tf

def Cys_environment(cysname, protname, dist_cutoff):

    tf.keras.backend.clear_session()
    #cmd.load(f"../bio_unit/{protname}","Prot") ### for importing local structures

    l= dist_cutoff #distance to the cystein
    
    cmd.fetch(protname, "Prot", type="pdb1")
    cmd.split_states("Prot")
    cmd.remove("Prot")
    Obj=cmd.get_object_list()
    
    list_results=[]

    y=Obj[0]
    CysSelect=(f"{y} and resname {cysname}") #### Selection for residue
    cmd.select('Cys', CysSelect)
    stored.list=[]
    cmd.iterate("(Cys & name sg)", "stored.list.append((resi,resn))");
#### Cysteines id
    if len(stored.list)==0:
        var=[]
        stored.amino=[]
        list_results=[]
        x=[]
        i=[]
    else:
        i=0
        while i<len(stored.list):
            x=stored.list[i][0]
            var =(f"(acc. and all) (and not resid {x}) nto. {l} of ({y} and resi {x} and n. sg)")
            #var =(f"br. all (and not resid {x}) nto. {l} of ({y} and resi {x} and n. sg)")
            cmd.select('Res', var)
            stored.amino=[]
            cmd.iterate("(Res)","stored.amino.append((resn, name))");
            list_results.append(stored.amino)
            i=i+1
            print(protname, list_results)
    list_results2=[*list_results]
    cmd.remove('all')
    #del l, var, x, i, list_results, CysSelect, stored.list, stored.amino, Obj
    #print(list_results2)
    #%store -z 
    return list_results2

In [3]:
def extract_env (AAList, Cys_name, dist_cutoff):
    tf.keras.backend.clear_session()
    L=[]
    Lcount=0
    for i in AAList:
        tf.keras.backend.clear_session()
        #print(i)
        L1=Cys_environment(Cys_name, i, dist_cutoff)
        Lcount=Lcount + len(L1)
        flat_list = [item for sublist in L1 for item in sublist]
        L.append(*[flat_list])
        del flat_list, L1
    LOut=[item for sublist in L for item in sublist]
    return Lcount, LOut
    tf.keras.backend.clear_session()

In [4]:
AminoAcids = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL','HOH', 'CSD', 'CSO', 'OCS', 'CSX']
backbone = ['N', 'O']
sidechains = ['OG', 'OG1', 'OE1', 'OE2', 'OD1', 'OD2', 'NE', 'NH2', 'NH1', 'ND1', 'NE2', 'NZ', 'ND2', 'OH', 'SG', 'SD']

textName=(f"OCS_Acc_results_BB1.txt")
text_file = open(textName, "w+")
#print('Res', 'N', 'O')
for j in backbone:
    for i in AminoAcids:
    # initializing substring
    # using list comprehension to get string with substring
        res1 = [i for i in L2 if j in i]
        L=f"{i}, {j}, {res1.count((i, j))}"
        text_file.write(L + '\n')
text_file.close()

textName=(f"OCS_Acc_results_SC1.txt")
text_file = open(textName, "w+")
for j in sidechains:
    for i in AminoAcids:
    # initializing substring
    # using list comprehension to get string with substring
        res1 = [i for i in L2 if j in i]
        L=f"{i}, {j}, {res1.count((i, j))}"
        text_file.write(L + '\n')
text_file.close()